## Imports

In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0
model_enwik = Word2vecMF()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read and preprocess enwik9

# Gensim

In [18]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [19]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [20]:
%%time
# Create word2vobject has no attributeec as matrix factorization model

'''model_enwik.data_to_matrices(real_sentences, dimension, 5, 
                             DB_to_file='enwik-200/matrices.npz',
                            indices_to_file='enwik-200/vocab.txt')'''
#sentences = load('data/x1')

model_enwik.data_to_matrices('data/x4', r=dimension, k=5, 
                             DB_to_file=False,
                            vocab_to_file=False)

Parsing sentences from training set
CPU times: user 8.86 s, sys: 236 ms, total: 9.1 s
Wall time: 9.06 s


In [21]:
#model_enwik.load_matrices('data/x1.npz')


In [22]:
#C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
C0, W0, step_size = SPPMI_init(model_enwik, dimension=dimension)
print(np.count_nonzero(C0))

/home/yerong/local/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


0.04052945679513818
813.1464918741565
(array([4.14720727, 2.22970812, 0.        , ..., 0.        , 0.        ,
       0.        ]), 'SPPMI')
(array([2.07944154, 0.        , 2.56494936, ...,       -inf,       -inf,
             -inf]), 'logD')
(array([-2.06776573, -2.22970812,  2.99282618, ..., -2.5106884 ,
       -2.42654372, -2.16823626]), 'logB')
('Initial loss', 27381368.617120914, 'theoretical step size', 1.175046174557565e-09)
303600


In [27]:
print(C0.shape[1])
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

3036


'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [28]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS',
               d=dimension,
               eta=1e-5,
               #eta = step_size,
               reg = regularization,
               MAX_ITER=5,
               from_iter=0,
               start_from='test',
               itv_print=1,
               itv_save=100000,
               init=(True, C0, W0), display=True,
               autostop=False)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/PSiter_fromtest_dim100_step1e-05_0.0
('Iter #:', 0, 'loss', 27381368.617120914)
('Iter #:', 1, 'loss', 27381368.617120914)
('Iter #:', 2, 'loss', 27376208.928626776)
('Iter #:', 3, 'loss', 27371114.124780748)
('Iter #:', 4, 'loss', 27366055.41602728)
('Iter #:', 5, 'loss', 27361003.099034976)
--- 27.689342498779297 seconds ---
